# Preamble

The purpose of this seminar is to introduce everyone to data manipulation and visualization in Python

Goals from this seminar:
- Know how to read in data from an excel sheet to Python
- Understand how to create simple publication-quality visualizations

*Not* goals from this seminar:
- How to write programs in Python
- Discussion of data structures/algorithms

# Step 1 - Setting up environment and modules

We are currently running a [Jupyter-Lab notebook](https://jupyter.org). This is essentially a environment where one can run code in a browser. This is one way to run code, but it is not the only way! 

We are using this format because it allows interactive, immediate, and reproducible visualizations directly from the browser!

Most scripts in Python use the below format to import modules.

Q: What are modules?
<details>
  <summary>Click for answer!</summary>
A: Modules are bits of code that people can create and share with others. Rather than reinvent the wheel everytime, we can  leverage the hard work and skills of others to help us! 
</details>


Here we import 2 modules. [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) and [seaborn](https://seaborn.pydata.org/). Click on both links and read a bit about them to understand what they do.

In [ ]:
# Step 1: Importing necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Step 2 - Finding and understanding our data

For this seminar, we will begin by looking at some data that was collected for a [study](https://www.cgdev.org/sites/default/files/9267_file_CGDWP95_Feb_2007.pdf) in 2006 by Clemens and Pettersson. 
Here is the abstract:
> The migration of doctors and nurses from Africa to rich countries has raised fears of an African **medical brain drain.** But empirical research on the issue has been hampered by lack of data. **How many doctors and nurses have left Africa? Which countries did they leave? Where have they settled?** To answer these questions we have compiled a dataset of the cumulative bilateral net flows of African-born physicians and nurses to the nine most important destination countries. It is the first dataset of bilateral migration flows specific to a skilled profession collected systematically for a large number of developing countries. This note makes the data available to the research community. 

They were kind enough to provide an excel spreadsheet of their data! You can download a copy [here](https://www.cgdev.org/sites/default/files/archive/doc/Data/Africa_health_emigration.xls)

Open up the excel spreadsheet and look through it. Note the format, layout, and different fields. What types of questions would you want to ask and answer with this data?

# Step 3 - Reading in the data

Let's read it in! Luckily `pandas` has an in-built method to read in excel spreadsheets. You can find the instructions (documentation) for it [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html).

Scroll down and look at the examples they have. Think about how they work and see if you can understand the basics.

In [ ]:
# Reading the excel file via the imported libraries
pd.read_excel(
    
    # identifying the path to the file
    "./african-health-emigration_clemens-pettersson-2006.xls",
)

What do you notice about the data. How is it read in and what are the issues you see with it?

Let's try reading it in again, this time we'll specify a header row

In [ ]:
doctor_data = pd.read_excel(
    "./african-health-emigration_clemens-pettersson-2006.xls",
    header=5, # The digit "5" indicates the index of the row to be considered as HEADER
)
doctor_data # Recalling the variable 

This looks better! Take a look at the `Total abroad` & `Frac.**` columns. How are they calculated? Also scroll to the bottom and see the rows past 53. How did the authors get these numbers?

*General rule of thumb* -- **Never** trust another person's calculations. You already have the raw data! Let's calculate these numbers ourselves. 

But first, let's get rid of the author's calculated areas.

In [ ]:
doctor_data = pd.read_excel(
    "./african-health-emigration_clemens-pettersson-2006.xls",
    header=5,
    usecols=[ # parameter indicating the columns to be used (of interest)
        "Sending country",
        "Domestic*",
        "UK",
        "USA",
        "France",
        "Canada",
        "Australia",
        "Portugal",
        "Spain",
        "Belgium",
        "So. Africa",
    ],
)
doctor_data = doctor_data.truncate(after=52) # after 52 because we want to remove everything after the 52th row
doctor_data

## Nice! Now let's compute the `Total abroad` and `Fraction` columns.

How would we learn how to do this? *Look it up!*

Q: How would I create a new column `Total abroad` by adding `UK - So. Africa`?
<details>
  <summary>Click for answer!</summary>

### Option 1:    
```
doctor_data['Total abroad'] = doctor_data['UK'] +
                                  doctor_data['USA'] +
                                  doctor_data['France'] +
                                  doctor_data['Canada'] +
                                  doctor_data['Australia'] +
                                  doctor_data['Portugal'] +
                                  doctor_data['Spain'] +
                                  doctor_data['Belgium'] +
                                  doctor_data['So. Africa']
```

### Option 2:                                  

```
doctor_data['Total abroad'] = doctor_data[[
                                    'UK', 
                                    'USA', 
                                    'France', 
                                    'Canada', 
                                    'Australia', 
                                    'Portugal',
                                    'Spain',
                                    'Belgium',
                                    'So. Africa'
                                ]].sum(axis=1)
```

### Option 3:                                

```
doctor_data['Total abroad'] = doctor_data[
                                    doctor_data.columns[
                                        ~doctor_data.columns.isin(
                                            ['Sending country','Domestic*']
                                        )
                                    ]
                                  ].sum(axis=1)
```
</details>

In [ ]:
doctor_data["Total abroad"] = doctor_data[
    [
        "UK",
        "USA",
        "France",
        "Canada",
        "Australia",
        "Portugal",
        "Spain",
        "Belgium",
        "So. Africa",
    ]
].sum(axis=1)

!!! What is going on?? Read the error message and think about how you might resolve this issue?

**_A good place to start is to identify if this works on a SMALL SCALE. Let's try making the sum just between French speaking countries_**

In [ ]:
doctor_data["Total abroad"] = doctor_data[["France", "Belgium"]].sum(axis=1) # setting the column components
doctor_data[["Sending country", "France", "Belgium", "Total abroad"]] # printing the columns 

Eureka!! This works! What if we add more countries...

In [ ]:
doctor_data["Total abroad"] = doctor_data[["UK", "USA", "France", "Belgium"]].sum(
    axis=1
) # adding UK and USA
doctor_data[["Sending country", "UK", "USA", "France", "Belgium", "Total abroad"]] # printing the results

This works! What if we add different countries...

In [ ]:
doctor_data["Total abroad"] = doctor_data[
    ["Spain", "Portugal", "So. Africa", "France", "Belgium"]
].sum(axis=1)
doctor_data[
    [
        "Sending country",
        "Spain",
        "Portugal",
        "So. Africa",
        "France",
        "Belgium",
        "Total abroad",
    ]
]

Aah, so this fails! When you narrow it down further you find the inclusion of `So. Africa` messes up the sum. Why might this be the case?

In [ ]:
doctor_data[["Sending country", "So. Africa"]]

Check out row 44, what is the value of that spot? It doesn't look like a number? Checking the excel file shows that they added a notation, but that's messing up the code. Let's replace that value with a number, and retry our sum code.

In [ ]:
doctor_data.at[44, "So. Africa"] = -834 # targetting specifically column: South Africa row: 44
doctor_data["Total abroad"] = doctor_data[
    [
        "UK",
        "USA",
        "France",
        "Canada",
        "Australia",
        "Portugal",
        "Spain",
        "Belgium",
        "So. Africa",
    ]
].sum(axis=1)
doctor_data

Lets now try to recompute the `Fraction` column... 

But wait, what data are we missing?

In [ ]:
# generating the total number of doctors in countries
doctor_data["Total doctors"] = doctor_data[[
    "Domestic*", "Total abroad"
]].sum(axis=1)
doctor_data

In [ ]:
# calculating the fraction of doctors abroad to the total doctors (if you want you can express this with percentages by multiplying the fraction by 100)
doctor_data["Fraction abroad"] = (
    doctor_data["Total abroad"] / doctor_data["Total doctors"]
)
doctor_data

**Biratangaje! We now have everything we need to make some PLOTS**

# Step 4 - Visualizing the data

Let's first try to recreate Figure 1 from their paper. They are using a bar plot to visualize the data, so let's [google how to make a bar plot in seaborn](https://letmegooglethat.com/?q=seaborn+bar+plot)

After carefully reading the documentation and looking at the examples, we land on something like this:

In [ ]:
# code of plotting the graphs
ax = sns.barplot(
    data=doctor_data,
    x="Fraction abroad",
    y="Sending country",
    #color=sns.color_palette()[0]
)

Looks a little goofy! How can we increase the size of the plot?

In [ ]:
ax = sns.barplot(
    doctor_data,
    x="Fraction abroad",
    y="Sending country",
)
ax.figure.set_size_inches((5, 9)) # 5 and 9 indicates the size of the plots in inches. Note: You can multiply by your numbers of choice

Ooo, much better. Now let's [sort the values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values) on the y-axis from highest fraction abroad to smallest

In [ ]:
# Visualizing the data as it pleases us (the desired order)
ax = sns.barplot(
    doctor_data.sort_values("Fraction abroad", ascending=True), # we mentioned the ascending to be false because we want to retain the descending order (desired order of view)
    x="Fraction abroad",
    y="Sending country",
)
ax.figure.set_size_inches((5, 9))

Looking good! Let's also change the x-axis to go from 0 - 1.

In [ ]:
# generating the plot similar to that one in the paper
ax = sns.barplot(
    doctor_data.sort_values("Fraction abroad", ascending=False),
    x="Fraction abroad",
    y="Sending country",
)
ax.figure.set_size_inches((5, 9))
ax.set(
    xlim=(0, 1), # increasing the limit of the x-axis
    title="Fraction of African-born physicians residing and working abroad circa 2000",
    xlabel="Fraction of physicians abroad, 2000",
    ylabel="",
)
sns.despine()

Congrats! We successfully remade the plot from the paper! 

But, what are some other questions we can answer with this data? What are other insights we can get from this dataset?

# Step 5 - Break down some code

In [ ]:
# What does the melt function do? How would you figure it out?
melted_doctor_data = pd.melt(
    doctor_data,
    id_vars=["Sending country", "Total doctors", "Total abroad"],
    value_vars=[
        "UK",
        "USA",
        "So. Africa",
        "France",
        "Belgium",
        "Spain",
        "Canada",
        "Portugal",
        "Australia",
    ],
    var_name="Receiving country",
    value_name="Number_going",
)

# What is going on here? What are the differences between 'Fraction_all_going' and 'Fraction_abroad_here'?
melted_doctor_data["Fraction_all_going"] = (
    melted_doctor_data["Number_going"] / melted_doctor_data["Total doctors"]
)
melted_doctor_data["Fraction_abroad_here"] = (
    melted_doctor_data["Number_going"] / melted_doctor_data["Total abroad"]
)

In [ ]:
# What can you estimate from this plot? What are some features you immediately notice?
ax = sns.barplot(
    # what is the loc command? What does it do?
    melted_doctor_data.loc[
        melted_doctor_data["Receiving country"].isin(["France", "Belgium"])
    ].sort_values("Fraction_abroad_here", ascending=False),
    x="Fraction_abroad_here",
    y="Sending country",
    hue="Receiving country",
)
ax.figure.set_size_inches((15, 9))
ax.set(
    xlabel="Fraction of abroad physicians in Francophone countries, 2000",
    ylabel="Country of Origin",
    xlim=(0, 1),
)
sns.despine()

In [ ]:
# What can you estimate from this plot? What are some features you immediately notice? what are your (overall) observations?
ax = sns.barplot(
    melted_doctor_data.sort_values("Fraction_abroad_here", ascending=False),
    x="Fraction_abroad_here",
    y="Sending country",
    hue="Receiving country",
)
ax.figure.set_size_inches((15, 9))
ax.set(
    xlabel="Fraction of abroad physicians, 2000",
    ylabel="Country of Origin",
    # what happens if I introduce the below commented code?
    # xlim=(0, 1),
)
sns.despine()

In [ ]:
# What can you estimate from this plot? What are some features you immediately notice? what are your (overall) observations?
ax = sns.barplot(
    melted_doctor_data[melted_doctor_data.sort_values("Fraction_abroad_here", ascending=False)['Sending country'].isin(
        ["Rwanda", "Burundi", "Seychelles", "Uganda", "Congo, DR"]
    )],
    x="Fraction_abroad_here",
    y="Sending country",
    hue="Receiving country",
)
ax.figure.set_size_inches((15, 9))
ax.set(
    xlabel="Fraction of abroad physicians, 2000",
    ylabel="Country of Origin",
    # what happens if I introduce the below commented code?
    # xlim=(0, 1),
)
sns.despine()

Let's drill down into *Rwanda*, I hear it's a pretty cool country 

How would we focus on the subset of data pertaining to Rwanda?

*Hint: Look at the above analysis and how we subset the data to only focus on Francophone receiving countries*

In [ ]:
rwanda_doctor_data = melted_doctor_data.loc[
    melted_doctor_data["Sending country"] == "Rwanda"
]
rwanda_doctor_data

Here's some code to get started. How would you make the plot better?

In [ ]:
sns.barplot(rwanda_doctor_data, x="Fraction_all_going", y="Receiving country")

In [ ]:
# for you to tweak!
sns.barplot(rwanda_doctor_data, x="Fraction_all_going", y="Receiving country")

# Step 6 - Try on your own!

Now that we've gone through one example, let's try generating similar plots from the other dataset, Nurses!

I've added some code to get you started!

In [ ]:
# hint: try adding header=5
nurse_data = pd.read_excel(
    "./african-health-emigration_clemens-pettersson-2006.xls",
    sheet_name="Nurses",
)
nurse_data

# Step 7 - Other data visualizations (Bonus)

Poke around the [Seaborn Examples](https://seaborn.pydata.org/examples/index.html) and try to imagine other creative ways to visualize the data.

In [ ]:
pivot_data = melted_doctor_data.pivot(
    index="Receiving country",
    columns="Sending country",
    values="Fraction_abroad_here",
).astype('float64')
pivot_data

In [ ]:
ax = sns.heatmap(pivot_data)
ax.figure.set_size_inches((9, 9))

In [ ]:
eastafrica = ["Burundi", "Comoros", "Djibouti", "Eritrea", "Ethiopia", "Kenya",
              "Mauritius", "Rwanda", "Seychelles", "Somalia","Sudan",
              "Tanzania", "Uganda"]
ax = sns.clustermap(pivot_data[eastafrica], cmap="mako", vmin=0, vmax=1)

Could I make these plots interactive? 
Yes! Check out [Plotly]() and [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)!